In [1]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmcv import Config
import copy
import os.path as osp
import os
import mmcv
import numpy as np
import time
import cv2
import matplotlib.pyplot as plt
import matplotlib
from pylab import *
from matplotlib import cm

def draw_features(width, height, x, savedir, savename):
    print("{}/{}".format(savename,width*height))
#     fig = plt.figure(figsize=(16, 16), frameon=False)
#     fig.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.95, wspace=0.05, hspace=0.05)
    for i in range(width * height):
#         plt.subplot(height,width, i + 1)
#         plt.axis('off')
        img = x[0, i, :, :]
#         img = img * -1
#         img = x
        pmin = np.min(img)
        pmax = np.max(img)
        img = ((img - pmin) / (pmax - pmin + 0.000001))*255  #float在[0，1]之间，转换成0-255
        img = np.where(img>100, img, 0)
        img=img.astype(np.uint8)  #转成unit8
        img=cv2.applyColorMap(img, cv2.COLORMAP_JET) #生成heat map
        img = img[:, :, ::-1]#注意cv2（BGR）和matplotlib(RGB)通道是相反的
        cv2.imwrite(savedir + '/' + str(i) + '_' + savename,mmcv.bgr2rgb(img),[int(cv2.IMWRITE_JPEG_QUALITY),95])
        
#     red = (0, 0, 255)
#     green = (0, 255, 0)
#     blue = (255, 0, 0)
#     cyan = (255, 255, 0)
#     yellow = (0, 255, 255)
#     magenta = (255, 0, 255)
#     white = (255, 255, 255)
#     black = (0, 0, 0)

os.environ["CUDA_VISIBLE_DEVICES"]="0"
cfg = Config.fromfile('./configs/faster_rcnn/xiaohei_faster_rcnn_r50_fpn_1x_chenzhou.py')
# cfg.work_dir = 'logs/xiaohei_faster_rcnn_r50_fpn_1x_chenzhou_202011050121/'
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_chenzhou.py')
# cfg = Config.fromfile('./configs/faster_rcnn/xiaohei_faster_rcnn_r50_fpn_1x_chenzhou.py')
cfg = Config.fromfile('./configs/mask_rcnn/mask_rcnn_r50_fpn_1x_chenzhou.py')
# cfg = Config.fromfile('./configs/cornernet/cornernet_hourglass104_mstest_8x6_210e_chenzhou.py')
cfg = Config.fromfile('./configs/ms_rcnn/ms_rcnn_r50_fpn_1x_chenzhou.py')
# cfg = Config.fromfile('/root/workspace/Thyroid_Solid_Nodule/code/yolo/mmdetection-master/configs/yolact/yolact_r50_1x8_chenzhou.py')
# cfg.work_dir = 'logs/faster_rcnn_r50_fpn_1x_chenzhou_202009291508/'
cfg.work_dir = 'logs/xiaohei_faster_rcnn_r50_fpn_1x_chenzhou_202009291506/'
# cfg.work_dir = 'logs/xiaohei_faster_rcnn_r50_fpn_1x_chenzhou_202010170903/'
cfg.work_dir = 'logs/mask_rcnn_r50_fpn_1x_chenzhou_202009300504/'
# cfg.work_dir = 'logs/cornernet_hourglass104_mstest_8x6_210e_chenzhou_202010111214/'
cfg.work_dir = 'logs/ms_rcnn_r50_fpn_1x_chenzhou_202010071352/'
# cfg.work_dir = '/root/workspace/Thyroid_Solid_Nodule/code/mmdetection/logs/yolact_r50_1x8_chenzhou_202010160301/'
cfg.seed = 0
cfg.total_epochs = 50
cfg.log_config.interval = 1000

checkpoint = cfg.work_dir + 'epoch_12.pth'

model = init_detector(cfg, checkpoint, device='cuda:0')
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES
# validations
result_path = cfg.work_dir + '/tests/'
text='ms:'

f = open('logs/cls.log', 'a+')
# os.linesep代表当前操作系统上的换行符

if os.path.exists(result_path + "/result/") == False:
    os.makedirs(result_path + "/result/")
for dir in range(0,5):
    if os.path.exists(result_path + "/heatmap/" + str(dir)) == False:
        os.makedirs(result_path + "/heatmap/" + str(dir))
    if os.path.exists(result_path + "/att_heatmap/" + str(dir)) == False:
        os.makedirs(result_path + "/att_heatmap/" + str(dir))
    if os.path.exists(result_path + "/rpn_heatmap/" + str(dir)) == False:
        os.makedirs(result_path + "/rpn_heatmap/" + str(dir))

path = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug/tests/'
for root, dirs, files in os.walk(os.path.abspath(path)):
    for file in files:
        img = mmcv.imread(path + file)
        (result, a, cls_score, bbox_pred), x = inference_detector(model, img)
#         print(file)
#         print(result[0])
#         print(cls_score.data.cpu().numpy())
#         print(bbox_pred.data.cpu().numpy().size)
        for index, feat in enumerate(cls_score.data.cpu().numpy()):
#             print(feat[0], feat[-1])
            n = np.exp(float(feat[0]))
            b = np.exp(float(feat[-1]))
            print(text, "%.8f" % (n/sum([n, b])), "%.8f" % (b/sum([n, b])))
            f.write(text + "%.8f" % (n/sum([n, b])) + os.linesep)

            

path = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug/validations/'
for root, dirs, files in os.walk(os.path.abspath(path)):
    for file in files:
        img = mmcv.imread(path + file)
        (result, a, cls_score, bbox_pred), x = inference_detector(model, img)
#         print(file)
#         print(result[0])
#         print(cls_score.data.cpu().numpy())
#         print(bbox_pred.data.cpu().numpy().size)
        for index, feat in enumerate(cls_score.data.cpu().numpy()):
#             print(feat[0], feat[-1])
            n = np.exp(float(feat[0]))
            b = np.exp(float(feat[-1]))
            print(text, "%.8f" % (n/sum([n, b])), "%.8f" % (b/sum([n, b])))
            f.write(text + "%.8f" % (n/sum([n, b])) + os.linesep)

            
# path = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou/image/'
# for root, dirs, files in os.walk(os.path.abspath(path)):
#     for file in files:
#         img = mmcv.imread(path + file)
#         (result, cls_score, bbox_pred), x = inference_detector(model, img)
# #         print(file)
# #         print(result[0])
# #         print(cls_score.data.cpu().numpy())
# #         print(bbox_pred.data.cpu().numpy().size)
#         for index, feat in enumerate(cls_score.data.cpu().numpy()):
# #             print(feat[0], feat[-1])
#             n = np.exp(float(feat[0]))
#             b = np.exp(float(feat[-1]))
#             print(text, "%.8f" % (n/sum([n, b])), "%.8f" % (b/sum([n, b])))
#             f.write(text + "%.8f" % (n/sum([n, b])) + "%.8f" % (b/sum([n, b])) + os.linesep)
f.close()

/root/workspace/Thyroid_Solid_Nodule/code/mmdetection/mmdet/apis/inference.py:41: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


loading annotations into memory...


/root/miniconda3/envs/mmdetection/lib/python3.6/site-packages/pycocotools/coco.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug/annotations/pseudo_images.json' mode='r' encoding='UTF-8'>
  dataset = json.load(open(annotation_file, 'r'))


Done (t=0.31s)
creating index...
index created!


/root/workspace/Thyroid_Solid_Nodule/code/mmdetection/mmdet/core/post_processing/bbox_nms.py:52: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629416375/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  labels = valid_mask.nonzero()[:, 1]


...

ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000008 0.99999992
ms: 0.00000000 1.00000000
ms: 0.00000002 0.99999998
ms: 0.00000000 1.00000000
ms: 0.00000786 0.99999214
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000003 0.99999997
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000002 0.99999998
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000008 0.99999992
ms: 0.000000

ms: 0.17099467 0.82900533
ms: 0.00226058 0.99773942
ms: 0.51967082 0.48032918
ms: 0.00132647 0.99867353
ms: 0.21575146 0.78424854
ms: 0.04746294 0.95253706
ms: 0.16095285 0.83904715
ms: 0.00007287 0.99992713
ms: 0.03853076 0.96146924
ms: 0.00216091 0.99783909
ms: 0.02285186 0.97714814
ms: 0.23506895 0.76493105
ms: 0.01168318 0.98831682
ms: 0.39735318 0.60264682
ms: 0.00115156 0.99884844
ms: 0.00027131 0.99972869
ms: 0.00002331 0.99997669
ms: 0.01482149 0.98517851
ms: 0.00002064 0.99997936
ms: 0.00000052 0.99999948
ms: 0.00069631 0.99930369
ms: 0.00021318 0.99978682
ms: 0.01995883 0.98004117
ms: 0.08138897 0.91861103
ms: 0.05028102 0.94971898
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.04059467 0.95940533
ms: 0.00000019 0.99999981
ms: 0.02292963 0.97707037
ms: 0.05126885 0.94873115
ms: 0.00003759 0.99996241
ms: 0.00023576 0.99976424
ms: 0.00000672 0.99999328
ms: 0.47137081 0.52862919
ms: 0.01207642 0.98792358
ms: 0.00488377 0.99511623
ms: 0.00000000 1.00000000
ms: 0.000000

ms: 0.00002011 0.99997989
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000002 0.99999998
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000006 0.99999994
ms: 0.00000048 0.99999952
ms: 0.00000000 1.00000000
ms: 0.00000001 0.99999999
ms: 0.00000000 1.00000000
ms: 0.00000041 0.99999959
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000001 0.99999999
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000370 0.99999630
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000012 0.99999988
ms: 0.00000000 1.00000000
ms: 0.00000001 0.99999999
ms: 0.00000011 0.99999989
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00001646 0.99998354
ms: 0.00000000 1.00000000
ms: 0.00000002 0.99999998
ms: 0.000000

ms: 0.99970125 0.00029875
ms: 0.99958912 0.00041088
ms: 0.99991296 0.00008704
ms: 0.00875192 0.99124808
ms: 0.99982522 0.00017478
ms: 0.99980943 0.00019057
ms: 0.99872482 0.00127518
ms: 0.99969792 0.00030208
ms: 0.02022222 0.97977778
ms: 0.99905857 0.00094143
ms: 0.00000069 0.99999931
ms: 0.00365415 0.99634585
ms: 0.00000917 0.99999083
ms: 0.00002767 0.99997233
ms: 0.34549679 0.65450321
ms: 0.80953476 0.19046524
ms: 0.98002993 0.01997007
ms: 0.00007736 0.99992264
ms: 0.00000022 0.99999978
ms: 0.00000024 0.99999976
ms: 0.00000006 0.99999994
ms: 0.00000000 1.00000000
ms: 0.00027647 0.99972353
ms: 0.00000000 1.00000000
ms: 0.99843483 0.00156517
ms: 0.99967150 0.00032850
ms: 0.91146040 0.08853960
ms: 0.99538805 0.00461195
ms: 0.99969763 0.00030237
ms: 0.00000000 1.00000000
ms: 0.00000022 0.99999978
ms: 0.00141417 0.99858583
ms: 0.00000018 0.99999982
ms: 0.66002560 0.33997440
ms: 0.00903512 0.99096488
ms: 0.00000120 0.99999880
ms: 0.00000319 0.99999681
ms: 0.00000000 1.00000000
ms: 0.000009

ms: 0.00000000 1.00000000
ms: 0.00000091 0.99999909
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000014 0.99999986
ms: 0.00000000 1.00000000
ms: 0.00000001 0.99999999
ms: 0.00000011 0.99999989
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000007 0.99999993
ms: 0.00000150 0.99999850
ms: 0.00000000 1.00000000
ms: 0.00000084 0.99999916
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000135 0.99999865
ms: 0.00000000 1.00000000
ms: 0.00000016 0.99999984
ms: 0.00000000 1.00000000
ms: 0.00000019 0.99999981
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000001 0.99999999
ms: 0.00000001 0.99999999
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00002239 0.99997761
ms: 0.00000006 0.99999994
ms: 0.00000004 0.99999996
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.00000000 1.00000000
ms: 0.000000

In [2]:
# mkdir -p demo/0 demo/1 demo/2 demo/3 demo/4
# cp 0/*A1B1C1D2E1_48THY1820180723105605185* demo/0/
# cp 1/*A1B1C1D2E1_48THY1820180723105605185* demo/1/
# cp 2/*A1B1C1D2E1_48THY1820180723105605185* demo/2/
# cp 3/*A1B1C1D2E1_48THY1820180723105605185* demo/3/
# cp 4/*A1B1C1D2E1_48THY1820180723105605185* demo/4/